In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
from mlxtend.frequent_patterns import association_rules, apriori, fpgrowth

from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.decomposition import TruncatedSVD
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
import platform
import re
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False 

if platform.system() == 'Darwin': 
    plt.style.use('seaborn-darkgrid') 
    rc('font', family = 'AppleGothic') 

In [38]:
!pwd

/home/jovyan/joohyun-nfs/jupyter/ABC_food/src


In [4]:
from glob import glob
# files = glob("./dataset/supply_chain/*.xls")
files = glob("../data/dataset/supply_chain/*.xls")
# files = glob("../data/dataset/supply_chain/2020.02.xls")
# joohyun-nfs/jupyter/ABC_food/data/dataset/supply_chain/2020.02.xls

In [5]:
df_list = [] 
for file in files:
    df_raw = pd.read_excel(file, "Table1", 
                      dtype={
                          'No':int,
                          '일자':str,
                          '매출처':str,
                          '코드':str,
                          '품목명': str,
                          '규격': str,
                          '원산지': str,
                          '단위': str,
                          '수량': float,
                          '단가': float,
                          '합계': float,
                          '금액': float,
                          '세액': float,
                          '비고': str                          
                      }
        )
    df_list.append(df_raw)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (6693331) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but S

In [6]:
df_raw = pd.concat(df_list)
df_raw.head()

,No,일자,매출처,코드,품목명,규격,원산지,단위,수량,단가,합계,금액,세액,비고
0,1,20200201,군산봉정요양병원**,1042594,(직) 세척당근 kg(생 2L 250~300g 중국산),특) kg 250~300g,중국산,Kg,3.0,1050.0,3150.0,3150.0,0.0,NaN
1,2,20200201,군산봉정요양병원**,1028277,(직) 흙대파 kg(생 끝 절단),kg(끝부분 절단),국내산,Kg,2.0,1950.0,3900.0,3900.0,0.0,NaN
2,3,20200201,군산봉정요양병원**,1004327,*요구르트 65ml(라이트 한국야쿠르트 냉장)..,(냉장)ea(65ml),국내산,ea,110.0,90.0,10890.0,9900.0,990.0,빨대주세요
3,4,20200201,군산봉정요양병원**,1049419,*)참기름 1.8L(쉐프초이스 신송),1.8L *신세계푸드급식브랜드,수입산(참깨분100％),통,1.0,13490.0,14839.0,13490.0,1349.0,NaN
4,5,20200201,군산봉정요양병원**,1038658,*)미향 1.8L(오뚜기)..,1.8L,국내,ea,1.0,3050.0,3355.0,3050.0,305.0,NaN


In [102]:
df_raw.shape

(872824, 14)

In [7]:
df_raw.fillna("", inplace=True)
df_raw.rename(columns={"일자":"day","매출처":"customer","코드":"code","수량":"quantity", "단가":"price", "합계":"total", "금액":"sum", "세액":"tax", "비고":"etc"}, inplace=True)
df_raw = df_raw[df_raw["etc"].str.contains("취소")==False]
df_raw = df_raw[df_raw["etc"].str.contains("반품")==False]
df_raw = df_raw[df_raw["quantity"]!=0]
df_raw = df_raw.drop(df_raw[df_raw["day"]=="소계"].index)
df_raw = df_raw.drop(df_raw[df_raw["day"]=="합계"].index)
print(df_raw.shape)

(857497, 14)


In [8]:
df = df_raw.copy()
df.drop(columns=["No","규격","단위","total","tax","etc"], inplace=True)
df.head()

,day,customer,code,품목명,원산지,quantity,price,sum
0,20200201,군산봉정요양병원**,1042594,(직) 세척당근 kg(생 2L 250~300g 중국산),중국산,3.0,1050.0,3150.0
1,20200201,군산봉정요양병원**,1028277,(직) 흙대파 kg(생 끝 절단),국내산,2.0,1950.0,3900.0
2,20200201,군산봉정요양병원**,1004327,*요구르트 65ml(라이트 한국야쿠르트 냉장)..,국내산,110.0,90.0,9900.0
3,20200201,군산봉정요양병원**,1049419,*)참기름 1.8L(쉐프초이스 신송),수입산(참깨분100％),1.0,13490.0,13490.0
4,20200201,군산봉정요양병원**,1038658,*)미향 1.8L(오뚜기)..,국내,1.0,3050.0,3050.0


In [9]:
# Number of raw data
df.rename(columns={"일자":"day","매출처":"customer","code":"product","품목명":"product_name","원산지":"origin", "수량":"quantity","단가":"price","금액":"aggregate"}, inplace=True) 
print(df.shape) 

(857497, 8)


In [107]:
df.head()

,day,customer,product,product_name,origin,quantity,price,sum
0,20211201,가은요양원,1028369,(직) 데친고사리 1kg(냉장},중국산,2.0,4030.0,8060.0
1,20211201,가은요양원,1105057,(직) 깐도라지채 1kg(냉장},중국산,2.0,7040.0,14080.0
2,20211201,가은요양원,1093670,콩나물 박스 4kg(상 곱슬이),"수입대두(중국,미국,캐나다)",1.0,5100.0,5100.0
3,20211201,가은요양원,1003475,*삼색수제비 1kg(동성 냉동),국내,2.0,2750.0,5500.0
4,20211201,가은요양원,1091376,*)자숙바지락살 350g(냉동),중국산,5.0,2420.0,12100.0


# Pearson correlation coefficients

In [10]:
# All customer from all dataset with TruncatedSVD

customer_quantity_product=df[['customer','quantity','product']]
customer_quantity_product

,customer,quantity,product
0,군산봉정요양병원**,3.0,1042594
1,군산봉정요양병원**,2.0,1028277
2,군산봉정요양병원**,110.0,1004327
3,군산봉정요양병원**,1.0,1049419
4,군산봉정요양병원**,1.0,1038658
...,...,...,...
33536,xx늘푸른요양병원,2.0,1028336
33537,xx늘푸른요양병원,9.0,1050467
33538,xx늘푸른요양병원,4.0,1087500
33540,xx늘푸른요양병원,1.0,1087488


In [11]:
rating_crosstab=customer_quantity_product.pivot_table(values='quantity', index='customer', columns='product', fill_value=0)
rating_crosstab.tail()

product,1000025,1000026,1000028,1000043,1000044,1000046,1000047,1000051,1000052,1000057,...,1160370,1160371,1160372,1160373,1160374,1160377,1160378,1160381,1160382,1164567
customer,,,,,,,,,,,,,,,,,,,,,
효_가족요양[행사],0.0,0.0,0,0.0,0,0.0,0,0,0,0,...,0,0,0.0,0.0,0.0,0,0,0,0,0
효사랑전주병원,0.0,0.0,0,0.0,0,0.0,0,0,0,0,...,0,0,0.0,0.0,0.0,0,0,0,0,0
효사랑전주병원[기타],0.0,0.0,0,0.0,0,0.0,0,0,0,0,...,0,0,0.0,0.0,0.0,0,0,0,0,0
효사랑전주병원[농산],0.0,0.0,0,0.0,0,0.0,0,0,0,0,...,0,0,0.0,0.0,0.0,0,0,0,0,0
효사랑전주병원[직원],0.0,0.0,0,0.0,0,0.0,0,0,0,0,...,0,0,0.0,0.0,0.0,0,0,0,0,0


In [12]:
X = rating_crosstab.T
X

customer,A-푸드마스터★,A-푸드마스터★[농산],ABC[배송팀],ABC[본사사무실],ABC[샘플 김장독],ABC[샘플 풍성김치],ABC[지원-등록],ABC[직원-구입],E-ABC[밴딩]이푸디,E-이푸디365 ★,...,해빛한방병원[직원],화원요양원,화원요양원[소모품],효_가족요양[본관],효_가족요양[직원],효_가족요양[행사],효사랑전주병원,효사랑전주병원[기타],효사랑전주병원[농산],효사랑전주병원[직원]
product,,,,,,,,,,,,,,,,,,,,,
1000025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1160378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1160381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
SVD = TruncatedSVD(n_components=12, random_state=5)
resultant_matrix = SVD.fit_transform(X)
# resultant_matrix
type(resultant_matrix)

numpy.ndarray

In [118]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

(23302, 23302)

col_idx = rating_crosstab.columns.get_loc("1028268")
corr_specific = corr_mat[col_idx]
corr=pd.DataFrame({'corr_specific':corr_specific, 'Products': rating_crosstab.columns}).sort_values('corr_specific', ascending=False)
corr=corr[corr['corr_specific']>=0.9]
corr

In [175]:
df[df['product']=='1028268']

,day,customer,product,product_name,origin,quantity,price,sum
253,20211201,군산정다운병원,1028268,(직) 취청오이 kg(생),국내산,3.0,5300.0,15900.0
335,20211201,군산효자요양병원**,1028268,(직) 취청오이 kg(생),국내산,1.0,5410.0,5410.0
386,20211201,노송천노인복지관[부식],1028268,(직) 취청오이 kg(생),국내산,2.0,5660.0,11320.0
553,20211201,더세움병원,1028268,(직) 취청오이 kg(생),국내산,3.0,5580.0,16740.0
1106,20211201,전북농업기술원,1028268,(직) 취청오이 kg(생),국내산,2.0,5300.0,10600.0
...,...,...,...,...,...,...,...,...
33015,20200831,해빛한방병원,1028268,(직) 취청오이 kg(생),국내산,1.0,3800.0,3800.0
33221,20200831,x만인요양병원,1028268,(직) 취청오이 kg(생),국내산,2.0,3800.0,7600.0
33252,20200831,xx늘푸른[5병동],1028268,(직) 취청오이 kg(생),국내산,1.0,3990.0,3990.0
33298,20200831,xx늘푸른요양병원,1028268,(직) 취청오이 kg(생),국내산,4.0,3990.0,15960.0


In [178]:
df[df['product']=='1000107']

,day,customer,product,product_name,origin,quantity,price,sum
34476,20201231,익산수재활요양병원**,1000107,*얼갈이배추 kg(생),국내산,1.0,0.0,0.0
3308,20201103,I쿡24[사랑방노인복지],1000107,*얼갈이배추 kg(생),국내산,2.0,2790.0,5580.0
3348,20201103,I쿡24[새힘주간센터],1000107,*얼갈이배추 kg(생),국내산,0.5,2790.0,1395.0
3380,20201103,I쿡24[엄마네집요양원],1000107,*얼갈이배추 kg(생),국내산,0.5,2790.0,1395.0
3435,20201103,I쿡24[정다운사랑의집],1000107,*얼갈이배추 kg(생),국내산,0.5,2790.0,1395.0
11732,20201110,I쿡24[새힘주간센터],1000107,*얼갈이배추 kg(생),국내산,0.5,2790.0,1395.0
11759,20201110,I쿡24[엄마네집요양원],1000107,*얼갈이배추 kg(생),국내산,0.5,2790.0,1395.0
1036,20210102,익산수재활요양병원**,1000107,*얼갈이배추 kg(생),국내산,1.0,1700.0,1700.0
7673,20210108,군산보현요양원,1000107,*얼갈이배추 kg(생),국내산,2.0,1700.0,3400.0
9882,20210109,청아람한방병원**,1000107,*얼갈이배추 kg(생),국내산,1.0,1540.0,1540.0


# apriori

In [48]:
# all data with apriori
onehot=customer_product.pivot_table(index='customer', columns='product', aggfunc=len, fill_value=0)
onehot=onehot>0

In [50]:
onehot

product,1000025,1000026,1000028,1000043,1000044,1000046,1000047,1000051,1000052,1000057,...,1160370,1160371,1160372,1160373,1160374,1160377,1160378,1160381,1160382,1164567
customer,,,,,,,,,,,,,,,,,,,,,
A-푸드마스터★,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
A-푸드마스터★[농산],False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABC[배송팀],False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABC[본사사무실],False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABC[샘플 김장독],False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
효_가족요양[행사],False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
효사랑전주병원,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
효사랑전주병원[기타],False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [57]:
frequent_itemsets = apriori(onehot, min_support = 0.05, max_len = 2, use_colnames=True)
rules = association_rules(frequent_itemsets)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1000107),(1089757),0.055215,0.196319,0.052147,0.944444,4.810764,0.041308,14.466258
1,(1000107),(1101063),0.055215,0.340491,0.052147,0.944444,2.773774,0.033347,11.871166
2,(1000107),(1107950),0.055215,0.285276,0.052147,0.944444,3.310633,0.036396,12.865031
3,(1000174),(1037609),0.138037,0.361963,0.116564,0.844444,2.332957,0.066600,4.101665
4,(1000174),(1041038),0.138037,0.254601,0.113497,0.822222,3.229451,0.078353,4.192868


In [181]:
# Indivisual customers with TruncatedSVD

gaun=df[df['customer']=='가은요양원']
gaun=gaun[['customer','quantity','product']]
gaun

,customer,quantity,product
0,가은요양원,2.0,1028369
1,가은요양원,2.0,1105057
2,가은요양원,1.0,1093670
3,가은요양원,2.0,1003475
4,가은요양원,5.0,1091376
...,...,...,...
29701,가은요양원,1.0,1093667
29702,가은요양원,2.0,1090164
29703,가은요양원,3.0,1028268
29704,가은요양원,1.0,1042775


In [216]:
gaun.value_counts(gaun['quantity']).describe()

count      17.000000
mean      176.058824
std       328.544797
min         1.000000
25%         4.000000
50%         6.000000
75%       114.000000
max      1032.000000
dtype: float64

In [275]:
rating_crosstab_gaun=gaun.pivot_table(values='quantity', index='customer', columns='product', fill_value=0)
rating_crosstab_gaun=rating_crosstab_gaun>0
rating_crosstab_gaun


product,1000121,1000172,1000181,1000183,1000202,1000207,1000221,1000227,1000237,1000299,...,1142613,1142614,1142856,1142859,1144042,1144682,1144710,1147718,1155578,1157181
customer,,,,,,,,,,,,,,,,,,,,,
가은요양원,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [276]:
frequent_itemsets_gaun = apriori(rating_crosstab_gaun, min_support = 0.1, max_len = 2, use_colnames=True)
rules_gaun = association_rules(frequent_itemsets_gaun)
rules_gaun

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1000121),(1000172),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(1000172),(1000121),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(1000121),(1000181),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(1000181),(1000121),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(1000121),(1000183),1.0,1.0,1.0,1.0,1.0,0.0,inf
...,...,...,...,...,...,...,...,...,...
266767,(1147718),(1155578),1.0,1.0,1.0,1.0,1.0,0.0,inf
266768,(1157181),(1147718),1.0,1.0,1.0,1.0,1.0,0.0,inf
266769,(1147718),(1157181),1.0,1.0,1.0,1.0,1.0,0.0,inf
266770,(1157181),(1155578),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [18]:
%time
rules_gaun_fp=fpgrowth(rating_crosstab_gaun, min_support=0.1, max_len=2,use_colnames=True)
rules_gaun_fp

CPU times: user 15 µs, sys: 2 µs, total: 17 µs
Wall time: 31.2 µs


,support,itemsets
0,1.0,(1157181)
1,1.0,(1037448)
2,1.0,(1036215)
3,1.0,(1036220)
4,1.0,(1036243)
...,...,...
133898,1.0,"(1081990, 1082441)"
133899,1.0,"(1081990, 1000121)"
133900,1.0,"(1082262, 1082441)"
133901,1.0,"(1000121, 1082262)"


In [278]:
rating_crosstab_gaun

product,1000121,1000172,1000181,1000183,1000202,1000207,1000221,1000227,1000237,1000299,...,1142613,1142614,1142856,1142859,1144042,1144682,1144710,1147718,1155578,1157181
customer,,,,,,,,,,,,,,,,,,,,,
가은요양원,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [280]:
from mlxtend.frequent_patterns import fpmax

In [303]:
%%time

fp=fpmax(rating_crosstab_gaun, min_support=1.0, max_len=None, use_colnames=True)
fp

CPU times: user 9.26 ms, sys: 4.35 ms, total: 13.6 ms
Wall time: 12.6 ms


,support,itemsets
0,1.0,"(1036287, 1021285, 1091376, 1036315, 1039822, ..."


In [19]:
from sklearn.neighbors import NearestNeighbors

In [34]:
X.index

Index(['1000025', '1000026', '1000028', '1000043', '1000044', '1000046',
       '1000047', '1000051', '1000052', '1000057',
       ...
       '1160370', '1160371', '1160372', '1160373', '1160374', '1160377',
       '1160378', '1160381', '1160382', '1164567'],
      dtype='object', name='product', length=23302)

In [36]:
X.head()

customer,A-푸드마스터★,A-푸드마스터★[농산],ABC[배송팀],ABC[본사사무실],ABC[샘플 김장독],ABC[샘플 풍성김치],ABC[지원-등록],ABC[직원-구입],E-ABC[밴딩]이푸디,E-이푸디365 ★,...,해빛한방병원[직원],화원요양원,화원요양원[소모품],효_가족요양[본관],효_가족요양[직원],효_가족요양[행사],효사랑전주병원,효사랑전주병원[기타],효사랑전주병원[농산],효사랑전주병원[직원]
product,,,,,,,,,,,,,,,,,,,,,
1000025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
knn=NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(X.values)
distances, indices=knn.kneighbors(X.values, n_neighbors=3)

In [29]:
indices

array([[    0,   254,   280],
       [    1, 19303,  2347],
       [    2,   371,  1005],
       ...,
       [  208,  1456,  1457],
       [ 1215,  2436,  4036],
       [18288, 18289, 18290]])

In [31]:
distances

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.13858234e-01, 3.38764823e-01],
       [2.22044605e-16, 2.92893219e-01, 2.92893219e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [41]:
index_for_X=X.index.tolist().index('1000025')
sim_X=indices[index_for_X].tolist()
X_distances=distances[index_for_X].tolist()
X_distances

[0.0, 0.0, 0.0]

In [65]:
# X.index.tolist().index('1000025')
# indices[0].tolist()
distances[0]

array([0., 0., 0.])

In [49]:
sim_X

[254, 280]

In [50]:
id_X=sim_X.index(index_for_X)
id_X

ValueError: 0 is not in list

In [51]:
index_for_X

0

In [52]:
sim_X.remove(index_for_X)

ValueError: list.remove(x): x not in list

In [53]:
X_distances

[0.0, 0.0, 0.0]

In [54]:
X_distances.pop()

0.0

In [55]:
sim_X

[254, 280]

In [56]:
X_distances

[0.0, 0.0]

In [115]:
x=X.copy()
number_neighbors = 3
knn=NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(x.values)
distances, indices=knn.kneighbors(x.values, n_neighbors=number_neighbors)
user_index=x.columns.tolist().index('효사랑전주병원[직원]')

for row, product in enumerate(x.index):
    if x.iloc[row, user_index]==0:
        sim_X=indices[row].tolist()
        X_distances=distances[row].tolist()
        
        if row in sim_X:
            id_X=sim_X.index(row)
            sim_X.remove(row)
            X_distances.pop(id_X)
            
        else:
            sim_X=sim_X[:number_neighbors-1]
            X_distances=X_distances[:number_neighbors-1]
            
        X_similatiry=[1-x for x in X_distances]
        X_similatiry_copy=X_similatiry.copy()
        nominator=0
        
        for s in range(0, len(X_similatiry)):
            if X.iloc[sim_X[s],user_index]==0:
                if len(X_similatiry_copy)==(number_neighbors-1):
                    X_similatiry_copy.pop(s)
                else:
                    X_similatiry_copy.pop(s-(len(X_similatiry)-len(X_similatiry_copy)))
            else:
                nominator=nominator+X_similatiry[s]*X.iloc[sim_X[s],user_index]
                
        if len(X_similatiry_copy)>0:
            if sum(X_similatiry_copy)>0:
                predicted_r=nominator/sum(X_similatiry_copy)
            else:
                predicted_r=0
        else:
            predicted_r=0
    else:
        predicted_r=0
        
    x.iloc[row,user_index]=predicted_r
        

In [140]:
def recommend_products(user,num_recommended_products):
    # print('The list of Product {} has purchased \n'.format(user))
    # for p in X[X[user]>0][user].index.tolist():
    #     print(p)
    # print('\n')
    
    recommend_products=[]
    
    for p in X[X[user]==0].index.tolist():
        index_X=X.index.tolist().index(p)
        predicted_rating=x.iloc[index_X, x.columns.tolist().index(user)]
        # if predicted_rating < 5.0 :
        #     continue
        # else:
        recommend_products.append((p, predicted_rating))
        
    sorted_rm=sorted(recommend_products, key=lambda x:x[1], reverse=True)
    
    print('The list of recommended Products \n')
    
    rank=1
    for recommend_products in sorted_rm[:num_recommended_products]:
        print('{}:{}-predicted rating:{}'.format(rank, recommend_products[0],recommend_products[1]))
        rank=rank+1

In [141]:
# sorted(customer_quantity_product['customer'].unique(), reverse=True)

In [142]:
%%time
recommend_products('효사랑전주병원[직원]',5)

The list of recommended Products 

1:1000470-predicted rating:10.0
2:1023281-predicted rating:10.0
CPU times: user 11.7 s, sys: 24.3 ms, total: 11.7 s
Wall time: 11.7 s


In [120]:
df[df['product']=='1000470']

,day,customer,product,product_name,origin,quantity,price,sum
2819,20200106,군산봉정요양병원**,1000470,노랑파프리카 kg(생 상 실속형**),국내산,0.5,5470.0,2735.0
4176,20200107,더세움병원,1000470,노랑파프리카 kg(생 상 실속형**),국내산,1.0,5470.0,5470.0
6272,20200109,우석대부속한방병원,1000470,노랑파프리카 kg(생 상 실속형**),국내산,1.0,5420.0,5420.0
9142,20200113,우석대부속한방병원,1000470,노랑파프리카 kg(생 상 실속형**),국내산,1.0,5800.0,5800.0
11693,20200116,누리푸드[김제한스],1000470,노랑파프리카 kg(생 상 실속형**),국내산,0.5,5940.0,2970.0
12276,20200116,함께사는마을,1000470,노랑파프리카 kg(생 상 실속형**),국내산,1.0,6400.0,6400.0
14614,20200120,더세움병원,1000470,노랑파프리카 kg(생 상 실속형**),국내산,0.5,5850.0,2925.0
20571,20200128,더세움병원,1000470,노랑파프리카 kg(생 상 실속형**),국내산,0.5,10550.0,5275.0
1340,20200702,군산봉정요양병원**,1000470,노랑파프리카 kg(생 상 실속형**),국내산,0.5,6770.0,3385.0
26810,20200724,청아람한방병원**,1000470,노랑파프리카 kg(생 상 실속형**),국내산,0.5,4460.0,2230.0


In [ ]:
# cut-off the number of product



In [143]:
df.head()

,day,customer,product,product_name,origin,quantity,price,sum
0,20211201,가은요양원,1028369,(직) 데친고사리 1kg(냉장},중국산,2.0,4030.0,8060.0
1,20211201,가은요양원,1105057,(직) 깐도라지채 1kg(냉장},중국산,2.0,7040.0,14080.0
2,20211201,가은요양원,1093670,콩나물 박스 4kg(상 곱슬이),"수입대두(중국,미국,캐나다)",1.0,5100.0,5100.0
3,20211201,가은요양원,1003475,*삼색수제비 1kg(동성 냉동),국내,2.0,2750.0,5500.0
4,20211201,가은요양원,1091376,*)자숙바지락살 350g(냉동),중국산,5.0,2420.0,12100.0


In [161]:
df['product'].value_counts().values

array([7318, 6215, 6103, ...,    1,    1,    1])

In [145]:
df['product'].quantile(q=0.5, interpolation='nearest')

'1044714'

In [146]:
df['product'].value_counts().describe()

count    23302.000000
mean        36.799288
std        183.944204
min          1.000000
25%          1.000000
50%          4.000000
75%         17.000000
max       7318.000000
Name: product, dtype: float64

In [174]:
df['product'].value_counts().values

array([7318, 6215, 6103, ...,    1,    1,    1])

In [212]:
temp=pd.DataFrame(df['product'].value_counts().values)
temp

,0
0,7318
1,6215
2,6103
3,6052
4,5064
...,...
23297,1
23298,1
23299,1
23300,1


In [218]:
# df.groupby('product').filter(lambda x:temp>17)
df.shape


(857497, 8)

In [227]:
# cut off the data by the frequency
df_over17=df[df['product'].map(df['product'].value_counts())>17].drop_duplicates(subset=['product'])
df_over17['product'].value_counts()

1028369    1
1038190    1
1043059    1
1090471    1
1087859    1
          ..
1000909    1
1019867    1
1104319    1
1106659    1
1122827    1
Name: product, Length: 5793, dtype: int64

In [304]:
df_over17

,day,customer,product,product_name,origin,quantity,price,sum
0,20211201,가은요양원,1028369,(직) 데친고사리 1kg(냉장},중국산,2.0,4030.0,8060.0
1,20211201,가은요양원,1105057,(직) 깐도라지채 1kg(냉장},중국산,2.0,7040.0,14080.0
2,20211201,가은요양원,1093670,콩나물 박스 4kg(상 곱슬이),"수입대두(중국,미국,캐나다)",1.0,5100.0,5100.0
3,20211201,가은요양원,1003475,*삼색수제비 1kg(동성 냉동),국내,2.0,2750.0,5500.0
4,20211201,가은요양원,1091376,*)자숙바지락살 350g(냉동),중국산,5.0,2420.0,12100.0
...,...,...,...,...,...,...,...,...
193,20200601,누리푸드[동전주우체국],1121673,공급중단) (할인) 토종닭닭다리살 1kg(냉동),국내산,4.0,3300.0,13200.0
445,20200601,사랑요양병원**,1077239,연두부 300g(맑은물에),수입산,2.0,390.0,780.0
13997,20210210,전주우리병원**,1041802,쑥 kg(생),국내산,0.5,14600.0,7300.0
975,20210302,실로암사랑요양병원**,1140217,*흙대파 kg(생 상),국산,4.0,8083.0,32332.0


In [228]:
gaun1=df_over17[df_over17['customer']=='가은요양원']
gaun1=gaun1[['customer','quantity','product']]
gaun1

,customer,quantity,product
0,가은요양원,2.0,1028369
1,가은요양원,2.0,1105057
2,가은요양원,1.0,1093670
3,가은요양원,2.0,1003475
4,가은요양원,5.0,1091376
3911,가은요양원,1.0,1087488
3912,가은요양원,3.0,1036267
3913,가은요양원,1.0,1079657
3917,가은요양원,1.0,1023500
3920,가은요양원,1.0,1028315


In [230]:
rating_crosstab_gaun1=gaun1.pivot_table(values='quantity', index='customer', columns='product', fill_value=0)
rating_crosstab_gaun1=rating_crosstab_gaun1>0
rating_crosstab_gaun1


product,1000221,1000310,1000673,1003475,1003790,1007697,1019206,1020200,1023500,1028315,...,1076920,1079657,1086549,1087488,1091376,1093670,1105057,1108636,1120197,1120272
customer,,,,,,,,,,,,,,,,,,,,,
가은요양원,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [231]:
frequent_itemsets_gaun1 = apriori(rating_crosstab_gaun1, min_support = 0.05, max_len = 2, use_colnames=True)
rules_gaun1 = association_rules(frequent_itemsets_gaun1)
rules_gaun1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1000221),(1000310),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(1000310),(1000221),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(1000221),(1000673),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(1000673),(1000221),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(1003475),(1000221),1.0,1.0,1.0,1.0,1.0,0.0,inf
...,...,...,...,...,...,...,...,...,...
547,(1120197),(1108636),1.0,1.0,1.0,1.0,1.0,0.0,inf
548,(1108636),(1120272),1.0,1.0,1.0,1.0,1.0,0.0,inf
549,(1120272),(1108636),1.0,1.0,1.0,1.0,1.0,0.0,inf
550,(1120197),(1120272),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [253]:
%%time
rules_gaun_fp1=fpgrowth(rating_crosstab_gaun1, min_support=0.7, max_len=3, use_colnames=True)
rules_gaun_fp1

CPU times: user 22.6 ms, sys: 117 µs, total: 22.7 ms
Wall time: 21.7 ms


,support,itemsets
0,1.0,(1120272)
1,1.0,(1120197)
2,1.0,(1000310)
3,1.0,(1000673)
4,1.0,(1003475)
...,...,...
2319,1.0,"(1000221, 1108636, 1091376)"
2320,1.0,"(1108636, 1093670, 1105057)"
2321,1.0,"(1000221, 1093670, 1105057)"
2322,1.0,"(1000221, 1108636, 1093670)"


In [240]:
rules_gaun_fp1

,support,itemsets
0,1.0,(1120272)
1,1.0,(1120197)
2,1.0,(1000310)
3,1.0,(1000673)
4,1.0,(1003475)
...,...,...
16777210,1.0,"(1087488, 1091376, 1028369, 1000310, 1086549, ..."
16777211,1.0,"(1087488, 1091376, 1028369, 1086549, 1105057, ..."
16777212,1.0,"(1087488, 1091376, 1028369, 1000310, 1086549, ..."
16777213,1.0,"(1087488, 1091376, 1028369, 1000310, 1086549, ..."


In [249]:
rules_gaun_fp1["item_count"] = rules_gaun_fp1["itemsets"].apply(lambda row : len(set(row)))

In [250]:
rules_gaun_fp1.tail()

,support,itemsets,item_count
16777210,1.0,"(1087488, 1091376, 1028369, 1000310, 1086549, ...",23
16777211,1.0,"(1087488, 1091376, 1028369, 1086549, 1105057, ...",23
16777212,1.0,"(1087488, 1091376, 1028369, 1000310, 1086549, ...",23
16777213,1.0,"(1087488, 1091376, 1028369, 1000310, 1086549, ...",23
16777214,1.0,"(1087488, 1091376, 1028369, 1000310, 1086549, ...",24


In [248]:
set(rules_gaun_fp1.iloc[-3]["itemsets"]) == set(rules_gaun_fp1.iloc[-2]["itemsets"])

False

In [234]:
rating_crosstab_gaun1.shape

(1, 24)

In [235]:
rating_crosstab_gaun1

product,1000221,1000310,1000673,1003475,1003790,1007697,1019206,1020200,1023500,1028315,...,1076920,1079657,1086549,1087488,1091376,1093670,1105057,1108636,1120197,1120272
customer,,,,,,,,,,,,,,,,,,,,,
가은요양원,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [237]:
import math
math.comb(24,2)

276

In [239]:
math.factorial(24)

620448401733239439360000

In [257]:
%%time
frequent_itemsets_gaun1 = apriori(rating_crosstab_gaun1, min_support = 0.7, max_len = 5, use_colnames=True)
rules_gaun1 = association_rules(frequent_itemsets_gaun1)
rules_gaun1

CPU times: user 3.39 s, sys: 383 ms, total: 3.78 s
Wall time: 3.77 s


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1000221),(1000310),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(1000310),(1000221),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(1000221),(1000673),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(1000673),(1000221),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(1003475),(1000221),1.0,1.0,1.0,1.0,1.0,0.0,inf
...,...,...,...,...,...,...,...,...,...
1436575,(1108636),"(1120197, 1120272, 1093670, 1105057)",1.0,1.0,1.0,1.0,1.0,0.0,inf
1436576,(1093670),"(1108636, 1120197, 1120272, 1105057)",1.0,1.0,1.0,1.0,1.0,0.0,inf
1436577,(1105057),"(1108636, 1120197, 1120272, 1093670)",1.0,1.0,1.0,1.0,1.0,0.0,inf
1436578,(1120197),"(1108636, 1120272, 1093670, 1105057)",1.0,1.0,1.0,1.0,1.0,0.0,inf


In [ ]:
# Split the basket by monthly after get 75% of the percentile as a criteria

In [306]:
from datetime import datetime

In [308]:
df.head()

,day,customer,product,product_name,origin,quantity,price,sum
0,20211201,가은요양원,1028369,(직) 데친고사리 1kg(냉장},중국산,2.0,4030.0,8060.0
1,20211201,가은요양원,1105057,(직) 깐도라지채 1kg(냉장},중국산,2.0,7040.0,14080.0
2,20211201,가은요양원,1093670,콩나물 박스 4kg(상 곱슬이),"수입대두(중국,미국,캐나다)",1.0,5100.0,5100.0
3,20211201,가은요양원,1003475,*삼색수제비 1kg(동성 냉동),국내,2.0,2750.0,5500.0
4,20211201,가은요양원,1091376,*)자숙바지락살 350g(냉동),중국산,5.0,2420.0,12100.0


In [309]:
df["day"] = pd.to_datetime(df["day"])
df["ym"] = df["day"].apply(lambda row : row.strftime("%Y%m"))
df.head()

,day,customer,product,product_name,origin,quantity,price,sum,ym
0,2021-12-01,가은요양원,1028369,(직) 데친고사리 1kg(냉장},중국산,2.0,4030.0,8060.0,202112
1,2021-12-01,가은요양원,1105057,(직) 깐도라지채 1kg(냉장},중국산,2.0,7040.0,14080.0,202112
2,2021-12-01,가은요양원,1093670,콩나물 박스 4kg(상 곱슬이),"수입대두(중국,미국,캐나다)",1.0,5100.0,5100.0,202112
3,2021-12-01,가은요양원,1003475,*삼색수제비 1kg(동성 냉동),국내,2.0,2750.0,5500.0,202112
4,2021-12-01,가은요양원,1091376,*)자숙바지락살 350g(냉동),중국산,5.0,2420.0,12100.0,202112


In [525]:
# cut off the data by lately 6months
months=str(int(max(df["ym"]))-int(5))
df_6months = df[df["ym"] >= months]
# df_6months.head()
df_6months.groupby(["ym"]).count()

,day,customer,product,product_name,origin,quantity,price,sum
ym,,,,,,,,
202107,44699,44699,44699,44699,44699,44699,44699,44699
202108,43298,43298,43298,43298,43298,43298,43298,43298
202109,41771,41771,41771,41771,41771,41771,41771,41771
202110,42485,42485,42485,42485,42485,42485,42485,42485
202111,45138,45138,45138,45138,45138,45138,45138,45138
202112,47906,47906,47906,47906,47906,47906,47906,47906


In [526]:
df_6months['product'].value_counts().describe()

count    14477.000000
mean        18.325413
std         72.484289
min          1.000000
25%          1.000000
50%          3.000000
75%         11.000000
max       2091.000000
Name: product, dtype: float64

In [527]:
quantile_75=(df_6months['product'].value_counts()).quantile(q=0.75)
quantile_75

11.0

In [528]:
# cut off the data by the frequency that is over 75% quantile
df_6months=df_6months[df_6months['product'].map(df_6months['product'].value_counts())>quantile_75].drop_duplicates(subset=['product'])
# df_6months_.head()
df_6months['product'].value_counts()
# total 3413 unique products

1028369    1
1089892    1
1084445    1
1083714    1
1004416    1
          ..
1004023    1
1048029    1
1096956    1
1087823    1
1094573    1
Name: product, Length: 3413, dtype: int64

In [720]:

df_6months.shape
df_6months

,day,customer,product,quantity,ym
0,2021-12-01,가은요양원,1028369,2.0,202112
1,2021-12-01,가은요양원,1105057,2.0,202112
2,2021-12-01,가은요양원,1093670,1.0,202112
3,2021-12-01,가은요양원,1003475,2.0,202112
4,2021-12-01,가은요양원,1091376,5.0,202112
...,...,...,...,...,...
449,2021-07-01,누리푸드[동전주우체국],1036517,6.0,202107
1885,2021-07-01,T토리앤쿡[2호 리싸이클링],1036484,9.0,202107
2747,2021-07-02,온고을재활의학과병원**,1104431,3.0,202107
3075,2021-07-02,전주학산요양병원,1093540,12.0,202107


In [955]:
# Put products into same basket by customer
unique_customer=df_6months.customer.unique()

for i in unique_customer:
    temp=df_6months[df_6months.customer==i]
    temp_list=temp['product'].to_list()
    df_6months.loc[temp.index, 'product_collection']=str(temp_list)

In [981]:
df_6months

,day,customer,product,quantity,ym,product_collection
0,2021-12-01,가은요양원,1028369,2.0,202112,"['1028369', '1105057', '1093670', '1003475', '..."
1,2021-12-01,가은요양원,1105057,2.0,202112,"['1028369', '1105057', '1093670', '1003475', '..."
2,2021-12-01,가은요양원,1093670,1.0,202112,"['1028369', '1105057', '1093670', '1003475', '..."
3,2021-12-01,가은요양원,1003475,2.0,202112,"['1028369', '1105057', '1093670', '1003475', '..."
4,2021-12-01,가은요양원,1091376,5.0,202112,"['1028369', '1105057', '1093670', '1003475', '..."
...,...,...,...,...,...,...
449,2021-07-01,누리푸드[동전주우체국],1036517,6.0,202107,"['1123806', '1005477', '1024942', '1142653', '..."
1885,2021-07-01,T토리앤쿡[2호 리싸이클링],1036484,9.0,202107,"['1039319', '1037398', '1039320', '1001893', '..."
2747,2021-07-02,온고을재활의학과병원**,1104431,3.0,202107,"['1046957', '1142646', '1036670', '1048752', '..."
3075,2021-07-02,전주학산요양병원,1093540,12.0,202107,"['1139835', '1020093', '1047399', '1082034', '..."


In [987]:
# And then put products into same basket monthly
groupby_ym=df_6months.groupby(['customer','ym'])['product'].unique()
groupby_ym=groupby_ym.to_frame().reset_index()

In [990]:
groupby_ym

,customer,ym,product
0,A-푸드마스터★,202108,[1098229]
1,A-푸드마스터★,202109,[1037858]
2,A-푸드마스터★,202110,"[1050255, 1084259, 1087336, 1082252]"
3,A-푸드마스터★,202111,"[1106669, 1095045, 1087337, 1093105, 1143472]"
4,A-푸드마스터★,202112,"[1146645, 1027158]"
...,...,...,...
418,효사랑전주병원,202111,"[1139017, 1095087]"
419,효사랑전주병원,202112,"[1007006, 1087763, 1087048, 1099212, 1020357, ..."
420,효사랑전주병원[농산],202111,[1119779]
421,효사랑전주병원[농산],202112,"[1119774, 1141897, 1119764, 1119785, 1119766, ..."


In [916]:
# for i in unique_customer:
#     temp=grouped[grouped.customer==i]
#     pro_list=temp['product']
#     grouped.loc[temp.index, 'group']=str([i.tolist() for i in pro_list])


In [998]:
grouped= groupby_ym.groupby('customer').agg(list)
# grouped_temp = grouped.groupby('customer')
grouped

,ym,product
customer,,
A-푸드마스터★,"[202108, 202109, 202110, 202111, 202112]","[[1098229], [1037858], [1050255, 1084259, 1087..."
A-푸드마스터★[농산],[202111],[[1094692]]
ABC[배송팀],[202112],"[[1019082, 1096008, 1019666]]"
ABC[본사사무실],[202112],"[[1106062, 1085162]]"
ABC[직원-구입],[202112],[[1062066]]
...,...,...
효_가족요양[본관],"[202109, 202110, 202111, 202112]","[[1026124], [1082411, 1154680], [1043051], [10..."
효_가족요양[행사],[202112],"[[1119732, 1046359, 1094962, 1000923, 1039701,..."
효사랑전주병원,"[202109, 202110, 202111, 202112]","[[1003094, 1006220, 1049221], [1003372, 104493..."


In [1008]:
groupby_ym['customer'].value_counts()

군산삼마요양원         6
군산요양병원[영양팀]     6
A-푸드마스터★        5
익산수재활요양병원**     5
구세군군산후생원[1]     5
               ..
군산효자요양병원[간식]    1
전주우리병원[직원]      1
전주은혜마을          1
전주지역자활[키트]      1
효사랑전주병원[직원]     1
Name: customer, Length: 176, dtype: int64

In [999]:
grouped['product']=grouped['product'].apply(lambda n:list(map(list, n)))


In [1001]:
foodmaster=grouped['product'][0]
foodmaster

[['1098229'],
 ['1037858'],
 ['1050255', '1084259', '1087336', '1082252'],
 ['1106669', '1095045', '1087337', '1093105', '1143472'],
 ['1146645', '1027158']]

In [1002]:
te = TransactionEncoder()
te_ary = te.fit(foodmaster).transform(foodmaster)
df_foodmaster = pd.DataFrame(te_ary, columns=te.columns_)
df_foodmaster

,1027158,1037858,1050255,1082252,1084259,1087336,1087337,1093105,1095045,1098229,1106669,1143472,1146645
0,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False,False
2,False,False,True,True,True,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,True,True,False,True,True,False
4,True,False,False,False,False,False,False,False,False,False,False,False,True


In [1]:
%%time
fpgrowth(df_foodmaster, min_support=0.1, use_colnames=True)

NameError: name 'fpgrowth' is not defined